archivos y librerias :p

In [5]:
import pandas as pd
import re

correo_file=r"..\1. Scrapeo\Contenido\correo.csv"
comercio_file=r"..\1. Scrapeo\Contenido\comercio.csv"
peru21_file=r"..\1. Scrapeo\Contenido\peru21.csv"


reemplazo_acentos = {'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u'}
patron_simb = r'[^a-záàâãéèêíìîóòôõúùûüñç\s]'
stopwords_es=[]
with open("spanish.txt","r",encoding="utf-8") as s:
    for i in s.readlines():
        stopwords_es.append(i.strip())



In [6]:
df_peru21=pd.read_csv(peru21_file,sep="&&&")
df_comercio=pd.read_csv(comercio_file,sep="&&&")
df_correo=pd.read_csv(correo_file,sep="&&&")

C:\Users\gonza\AppData\Local\Temp\ipykernel_13916\2125926350.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_peru21=pd.read_csv(peru21_file,sep="&&&")
C:\Users\gonza\AppData\Local\Temp\ipykernel_13916\2125926350.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_comercio=pd.read_csv(comercio_file,sep="&&&")
C:\Users\gonza\AppData\Local\Temp\ipykernel_13916\2125926350.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying e

In [4]:
df_peru21.fillna("",inplace=True)
df_peru21['dia']=pd.to_datetime(df_peru21['dia'].str.strip(),format='%d-%m-%Y')
df_peru21[df_peru21['cuerpo'].isna()]

,link,headline,bajada,antetitulo,cuerpo,dia,etiquetas


In [5]:
df_comercio.fillna("",inplace=True)
df_comercio['dia']=pd.to_datetime(df_comercio['dia'].str.strip(),format='%d-%m-%Y')
df_comercio[df_comercio['cuerpo'].isna()]

,link,headline,bajada,antetitulo,cuerpo,dia,etiquetas


In [6]:
df_correo.fillna("",inplace=True)
df_correo['dia']=pd.to_datetime(df_correo['dia'].str.strip(),format='%d-%m-%Y')
df_correo[df_correo['cuerpo'].isna()]

,link,headline,bajada,antetitulo,cuerpo,dia,etiquetas


### Estandarizacion base

- minusculas
- puntuacion
- stopwords
- tokenizacion
- lematizacion

In [14]:
import re


def preprocesamiento(texto):
    limpio = str(texto).lower()
    
    limpio = re.sub(r'http\S+', '', limpio)  # Elimina URLs
    limpio = re.sub(r'pic\.twitter\.com/\S+', '', limpio) 
    # Reemplazar caracteres acentuados
    limpio = re.sub(patron_simb," ",limpio,flags=re.IGNORECASE)
    limpio = re.sub(r'[áéíóú]', lambda x: reemplazo_acentos[x.group()], limpio)

    # Tokenización y eliminación de puntuación
    tokens = limpio.split()
    
    # Eliminación de stopwords
    tokens_sin_stopwords = [word for word in tokens if word not in stopwords_es]
    
    # Filtrar tokens que sean números
    tokens_sin_numeros = [word for word in tokens_sin_stopwords if not re.match(r'^\d+$', word)]

    limpio = " ".join(tokens_sin_numeros)
    
    return limpio



In [24]:
campos=['headline','bajada','cuerpo']
df_test=df_comercio[:70000]

In [25]:
for i in range(70000):
    body=df_test['cuerpo'].loc[42]
    limpio=preprocesamiento(body)
    if "_" in limpio:
        print(body)
        print(limpio)

## Comercio

In [26]:
com_lim = df_comercio.copy()

for col in campos:
    com_lim[col] = df_comercio[col].apply(preprocesamiento)
com_lim.head()

In [27]:
com_lim.to_csv('comercio_df1.txt', index=False)

## Correo

In [28]:
crr_lim=df_correo.copy()
for i in campos:
    crr_lim[i] = crr_lim[i].fillna("")
    crr_lim[i] = df_correo[i].apply(preprocesamiento)
crr_lim.head()

,link,headline,bajada,antetitulo,cuerpo,dia,etiquetas
0,https://diariocorreo.pe/mundo/salva-de-morir-d...,salva morir varias horas rodeado tiburones,pescador fortuna ser rescatado tras caer bote ...,Mundo,pescador sobrevivio horas aguas frias infestad...,2017-01-03,"Pescador, Austrlaia"
1,https://diariocorreo.pe/edicion/moquegua/presi...,presidente fiscales promete lucha frontal corr...,asi mismo anuncio gestionara construccion morg...,Moquegua,nuevo presidente junta fiscales superiores moq...,2017-01-03,"Fiscalía, Corrupción, Moquegua"
2,https://diariocorreo.pe/economia/pequenas-empr...,pequeñas empresas pueden crearse red notarios,asociacion emprendedores peru señala genera co...,Economía,tal hace chile mexico peru podria constituir e...,2017-01-03,
3,https://diariocorreo.pe/peru/defensoria-pide-s...,defensoria pide suspender alza costo peaje pue...,traves comunicado institucion señalo residente...,Perú,atencion denuncias excesivo incremento costo p...,2017-01-03,Defensoría del Pueblo
4,https://diariocorreo.pe/edicion/moquegua/tempo...,temporal lluvias tormentas asi region moquegua...,ubinas reporto restriccion via activacion queb...,Moquegua,lluvias moderada intensidad registran zona and...,2017-01-03,"Lluvias, Moquegua, Indeci"


In [29]:
crr_lim.to_csv('correo_df1.txt', index=False)

## Peru21

In [30]:
P21_lim=df_peru21.copy()
for i in campos:
    P21_lim[i] = P21_lim[i].fillna("")
    P21_lim[i] = df_peru21[i].apply(preprocesamiento)
P21_lim.head()

,link,headline,bajada,antetitulo,cuerpo,dia,etiquetas
0,https://peru21.pe/peru/caso-arlette-fallas-fal...,caso arlette contreras contamos detalles falla...,expertos afirman jueces pudieron aplicar delit...,Perú,pasado viernes adriano pozo agresor vimos pant...,2018-02-20,"Feminicidio, Violencia física, Adriano Pozo, N..."
1,https://peru21.pe/politica/ivan-noguera-dr-roc...,ivan noguera dr rock reconozco peru impotente,ex intengrante consejo nacional magistratura a...,Política,tremendo show ivan noguera ex integrante conse...,2018-10-04,"Consejo Nacional de la Magistratura, Dr. Rock..."
2,https://peru21.pe/fotogalerias/whatsapp-aplica...,partir ahora whatsapp dejara funcionar smartph...,whatsapp decidio funcionar moviles considerari...,Fotogalerías,atencion usuarios whatsapp aplicacion dejara b...,2018-01-01,"Smartphones, BlackBerry, Whatsapp"
3,https://peru21.pe/politica/indulto-alberto-fuj...,alberto quintanilla indulto fujimori espero pp...,vocero nuevo peru indico rpp encuentran sorpre...,Política,luego conocerse presidente pedro pablo kuczyns...,2017-12-24,"Alberto Fujimori, PPK, Alberto Quintanilla, In..."
4,https://peru21.pe/peru/hospital-belen-trujillo...,hospital belen trujillo presenta hacinamiento ...,susalud detecto pacientes derivados uci perman...,Perú,alan benitesel hospital belen trujillo present...,2017-10-25,"La Libertad, Hospital Regional de Trujillo"


In [31]:
P21_lim.to_csv('peru21_df1.txt', index=False)